In [87]:
import os
from dotenv import load_dotenv
import openai
import requests
import json
import pandas as pd

In [16]:
load_dotenv()
CB_API_KEY = os.getenv('CB_API_KEY')
OPENAI_API_KEY = os.getenv('OPEN_AI_KEY')

In [88]:
fake_news_articles = pd.read_csv("data/fake-news/train.csv")
fake_news_articles

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [22]:
article_content = 'WILMINGTON, Del. — Dominion Voting Systems’ $1.6 billion defamation lawsuit against Fox News has been delayed on the eve of its scheduled start date, an official for the court said Sunday.The conclusion of the jury selection process and the start of opening statements, which were set for Monday morning, have been pushed to Tuesday morning at 9 a.m. ET, according to a statement from Delaware Superior Court.Spokespeople for Fox and Dominion did not immediately provide statements when asked for comment after the delay was announced Sunday night.Dominion, which sells voting machines and election software, claimed it was defamed by Fox Corp. and its cable TV networks after Fox aired false claims that the company had rigged the 2020 election against former President Donald Trump.Fox has argued that Dominion has not met the legal standard for defamation, and that the statements made about Dominion on its air were protected by the First Amendment.The Wall Street Journal reported earlier Sunday that Fox has made a last-minute push to settle the lawsuit out of court.Most defamation cases settle out of court. But as recently as Saturday, the possibility of Dominion and Fox avoiding trial seemed as unlikely as ever. “In the coming weeks, we will prove Fox spread lies causing enormous damage to Dominion. We look forward to trial,” a Dominion spokesman told CNBC on Saturday morning.Judge Eric Davis, who is presiding over the case, said in a statement from the court that he will formally announce the delay from his courtroom on Monday at 9 a.m. ET.'

In [144]:
article_content = fake_news_articles.iloc[0]["text"]

In [145]:
article_content

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [154]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text, usually an article. You will always return a numbered list and nothing else. Each item in the numbered list will always start with the topic of the article."},
        {"role": "user", "content": "Give me a numbered list of the most important facts and claims relevant to the argument of the following article. Make sure that each item in the list is a sentence that contains the fact or claim and any context from the article needed to fact check it. Also give me the argument of the article in a few words."},
        {"role": "user", "content": article_content}
    ]
)

In [160]:
response_content = response["choices"][0]["message"]["content"]
topic = response_content[response_content.find("Argument: ") + 10:response_content.find("\n")]
facts = response_content[response_content.find("\n") + 2:]
facts = [fact[fact.find(" ") + 1:] for fact in facts.split("\n")]
topic, facts

("Democrats on relevant committees knew about the FBI's investigation into Clinton's emails when Republicans did, through a tweet by the Oversight Committee's chairman, Jason Chaffetz.",
 ["Democrats on the relevant committees did not receive Comey's letter about the FBI's investigation into Clinton's emails until after the Republican Chairmen did, and only received it after Jason Chaffetz tweeted about it and made it public.",
  'The FBI director gave advance notice of this letter to Chaffetz and other Republicans, and they\xa0did not let their Democratic counterparts know about it.',
  "acted irresponsibly by not notifying ranking member Elijah Cummings about something as explosive as the news of Comey's letter.",
  'violates the basic standards of fairness.'])

In [156]:
response

<OpenAIObject chat.completion id=chatcmpl-76AOYefvedKtHVQZmcp9jSQiHTYRj at 0x1a06c697360> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Argument: Democrats on relevant committees knew about the FBI's investigation into Clinton's emails when Republicans did, through a tweet by the Oversight Committee's chairman, Jason Chaffetz.\n1. Democrats on the relevant committees did not receive Comey's letter about the FBI's investigation into Clinton's emails until after the Republican Chairmen did, and only received it after Jason Chaffetz tweeted about it and made it public.\n2. The FBI director gave advance notice of this letter to Chaffetz and other Republicans, and they\u00a0did not let their Democratic counterparts know about it.\n3.\u00a0Chaffetz acted irresponsibly by not notifying ranking member Elijah Cummings about something as explosive as the news of Comey's letter.\n4.\u00a0Chaffetz's\u00a0behavior violates the b

In [101]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/score/text/{topic + '; ' + facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'version': '2', 'claim': "A House Democratic aide claims they did not receive Comey's letter regarding an investigation of Clinton's emails until after the letter had been tweeted by a Republican committee chairman. ; The ranking Democrats on relevant committees did not hear about Comey's letter from Comey himself, but from a tweet from a Republican committee chairman, according to a senior House Democratic aide. ", 'results': [{'text': "A House Democratic aide claims they did not receive Comey's letter regarding an investigation of Clinton's emails until after the letter had been tweeted by a Republican committee chairman. ; The ranking Democrats on relevant committees did not hear about Comey's letter from Comey himself, but from a tweet from a Republican committee chairman, according to a senior House Democratic aide. ", 'index': 0, 'score': 0.7900983883}]}


In [98]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/knowledge_bases/{facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'claim': "The ranking Democrats on relevant committees did not hear about Comey's letter from Comey himself, but from a tweet from a Republican committee chairman, according to a senior House Democratic aide. ", 'origin': 'Claim Checker - Knowledge Bases', 'justification': []}


In [99]:
api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/fact_matcher/{facts[0]}"
request_headers = {"x-api-key": CB_API_KEY}

api_response = requests.get(url=api_endpoint, headers=request_headers)

print(api_response.json())

{'claim': "The ranking Democrats on relevant committees did not hear about Comey's letter from Comey himself, but from a tweet from a Republican committee chairman, according to a senior House Democratic aide. ", 'origin': 'Claim Matcher', 'justification': []}


In [161]:
def score_facts_for_validation(facts):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/score/text/sentences/{' '.join(facts)}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['results']

def get_justification(fact):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/knowledge_bases/{fact}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['justification']

def get_matching_facts(fact):
    api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/query/fact_matcher/{fact}"
    request_headers = {"x-api-key": CB_API_KEY}

    api_response = requests.get(url=api_endpoint, headers=request_headers)

    return api_response.json()['justification']
    
def get_topic_and_claims(article_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are fact and claim detector responsible for extracting the most important facts and claims from a piece of text, usually an article. You will always return a numbered list and nothing else. Each item in the numbered list will always start with the topic of the article."},
            {"role": "user", "content": "Give me a numbered list of the most important facts and claims relevant to the argument of the following article. Make sure that each item in the list is a sentence that contains the fact or claim and any context from the article needed to fact check it. Also give me the argument of the article in a few words."},
            {"role": "user", "content": article_content}
        ]
    )
    
    response_content = response["choices"][0]["message"]["content"]
    topic = response_content[response_content.find("Argument: ") + 10:response_content.find("\n")]
    facts = response_content[response_content.find("\n") + 2:]
    facts = [fact[fact.find(" ") + 1:] for fact in facts.split("\n")]
    return topic, facts

def validate_facts(topic, facts):
    topic_prefixed_facts = [topic[:-1] + "; " + fact for fact in facts]
    facts = [topic] + facts
    topic_prefixed_facts = [topic] + facts
    
    scores = score_facts_for_validation(topic_prefixed_facts)
    
    print(f"Topic: {topic}")
    for i in range(len(facts)):
        print("------------------------")
        print(f"Fact: {facts[i]}")
        print(f"Score: {scores[i]['score']}")
        if scores[i]['score'] > 0.5:
            justification = get_justification(facts[i])
            print(f"Justification: {justification}")
            matches = get_matching_facts(facts[i])
            print(f"Matches: {matches}")

In [168]:
topic, facts = get_topic_and_claims(article_content)

In [169]:
validate_facts(topic, facts)

Topic: The article is a transcript of a Republican debate where various candidates answer questions and present their political stances.
------------------------
Fact: The article is a transcript of a Republican debate where various candidates answer questions and present their political stances.
Score: 0.2396263949
------------------------
Fact: Jeb Bush claims he applied conservative principles in a purple state effectively and created 1.3 million jobs.
Score: 0.2396263949
------------------------
Fact: Megyn Kelly confronts Donald Trump for making several disparaging comments about women's looks, including a comment about a Celebrity Apprentice contestant being on her knees. Kelly asks whether he has the temperament to be president and how he'll defend against charges from Hillary Clinton that he's part of the "war on women." Trump later argues that the country has more important things to worry about than political correctness.
Score: 0.8699555213
Justification: []
Matches: []
----